Attempt to apply a classifier into the data.

In [1]:
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'assignment_2'))
	print(os.getcwd())
except:
	pass

c:\Users\Renjie Zhu\Documents\workspace\recommender_system\assignment_2


In [2]:
import numpy as np 
import pandas as pd 
import pprint

pp = pprint.PrettyPrinter(indent=4)

def train_test_validation(closed_data, train_frac=0.8):
    """ 8:1:1 train, text ,validation split """
    train = closed_data.sample(frac=train_frac)
    remain = closed_data.drop(train.index)
    test = remain.sample(frac=0.5)
    validation = remain.drop(test.index)
    return train, test, validation

In [3]:
# Data Preprocessing
closed_data = pd.read_csv("closed_data.csv")
closed_data = closed_data.loc[closed_data["district"].isin([1,2,3,4,5,6,7,8,9])]
print(f"Dataset size is {len(closed_data)}.")

Dataset size is 125177.


In [4]:
# Select columns
'''
Preprocessing:
	- Keep only closed case
	- Keep case with district data

Features:
	- Service name: service_name, onehot
	- Total population: total_pop2017
	- White population: white_pop2017
	- African American population: afram_pop2017
	- Hispanic population population: hisp_pop2017
	- Case origion: case_origin, onehot
	- Description sentiment: feature sentiment analysis, homebrewed
        - neg, neu, com, pos
    - Average income: avg_agi
'''
column_lst = ["total_pop2017", "white_pop2017", "afram_pop2017", "hisp_pop2017",\
              "avg_agi", "neg", "neu", "com", "pos"]
onehot_lst = ["service_name", "case_origin"]
prep_data = closed_data[column_lst]

In [10]:
# Onehot Process
onehot_data = closed_data[onehot_lst]
categorize_data = pd.get_dummies(onehot_data, prefix=onehot_lst)
logistic_label = closed_data["lncase_age_days"] <= 1.0
dataset = pd.concat([prep_data, categorize_data], axis=1)
dataset["logis_label"] = logistic_label.astype(int)

In [13]:
dataset["logis_label"].describe()

count    125177.000000
mean          0.230026
std           0.420851
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: logis_label, dtype: float64

In [12]:
train, test, validation = train_test_validation(dataset)

## naive logistic regression (log(casetime)<1)